# VIBE: Video Inference for Human Body Pose and Shape Estimation

Demo of the original PyTorch based implementation provided here: https://github.com/mkocabas/VIBE

## Note
Before running this notebook make sure that your runtime type is 'Python 3 with GPU acceleration'. Go to Edit > Notebook settings > Hardware Accelerator > Select "GPU".

## More Info
- Paper: https://arxiv.org/abs/1912.05656
- Repo: https://github.com/mkocabas/VIBE

In [ ]:
# Clonar el repositorio oficial de VIBE en /content/VIBE
# Clone the repo
!git clone https://github.com/mkocabas/VIBE.git

In [ ]:
# Instalar dependencias específicas que requiere el repositorio VIBE (Torch antiguo, NumPy y otras)
# Estas versiones son las recomendadas por el repositorio original.
# Install the other requirements
!pip install torch==1.4.0 numpy==1.17.5
!pip install git+https://github.com/giacaglia/pytube.git --upgrade
!pip install -r requirements.txt

In [ ]:
# Descargar los pesos preentrenados de VIBE y los datos del modelo SMPL necesarios para la demo
# Download pretrained weights and SMPL data
!source scripts/prepare_data.sh

### Run the demo code.

Check https://github.com/mkocabas/VIBE/blob/master/doc/demo.md for more details about demo.

**Note:** Final rendering is slow compared to inference. We use pyrender with GPU accelaration and it takes 2-3 FPS per image. Please let us know if you know any faster alternative.

En las celdas siguientes se ejecutará la demo usando un video propio (video_salida.mp4) y se generará el archivo vibe_output.pkl con las poses 3D.

In [ ]:
# Instalación de dependencias adicionales de VIBE:
# - multi-person-tracker, yolov3, filterpy, yacs: detección y seguimiento de personas
# - smplx, trimesh, pyrender, chumpy: modelo de cuerpo 3D y renderizado
!pip install git+https://github.com/mkocabas/multi-person-tracker.git

In [ ]:
!pip install filterpy

In [ ]:
!pip install git+https://github.com/mkocabas/yolov3-pytorch.git

In [ ]:
!python -m pip install yacs

In [ ]:
!pip install smplx

In [ ]:
!pip install trimesh

In [ ]:
!pip install pyrender

In [ ]:
!pip install chumpy

In [ ]:
# Ejecutar la demo de VIBE sobre el video de entrada y generar resultados en la carpeta output/
# (incluye 'output/video_salida/vibe_output.pkl' con las poses 3D por frame)
# Run the demo
!python demo.py --vid_file /content/video_salida.mp4 --output_folder output/ --sideview

# Comprimir y descargar todos los resultados (video renderizado y archivo vibe_output.pkl)
# Para otros notebooks se asumirá que este PKL está disponible (p.ej. como /content/vibe_output.pkl)
# Descarga automática
!zip -r output.zip output/
from google.colab import files
files.download('output.zip')
# You may use --sideview flag to enable from a different viewpoint, note that this doubles rendering time.
# !python demo.py --vid_file sample_video.mp4 --output_folder output/ --sideview

# You may also run VIBE on a YouTube video by providing a link
# python demo.py --vid_file https://www.youtube.com/watch?v=c4DAnQ6DtF8 --output_folder output/ --display

In [ ]:
# Reproducir el video generado por VIBE (solo para inspección visual, no afecta al pipeline de métricas)
# Play the generated video
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=500 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('output/video_salida/video_salida_vibe_result.mp4')

In [ ]:
# Inspeccionar el contenido de 'vibe_output.pkl': verificar claves y tamaños de los arrays que usaremos después
# Inspect the output file content
import joblib
output = joblib.load('output/video_salida/vibe_output.pkl')
print('Track ids:', output.keys(), end='\n\n')

print('VIBE output file content:', end='\n\n')
for k,v in output[1].items():
  if k != 'joints2d':
    print(k, v.shape)

In [ ]:
# === Plot 3D joints for first 10 frames with Matplotlib ===
# Visualización de los joints3d (49 articulaciones) de VIBE en 3D para algunos frames de ejemplo.
# Esto sirve solo para comprobar que las poses estimadas son razonables.

import joblib
import numpy as np
import matplotlib.pyplot as plt

# Ruta al output de VIBE
pkl_path = 'output/video_salida/vibe_output.pkl'

# Cargar
output = joblib.load(pkl_path)
track_id = next(iter(output.keys()))  # o usa 1 si lo sabes fijo
J = output[track_id]['joints3d']      # (T, 49, 3)

# Opcional: centrar en pelvis (asumiendo pelvis = índice 0)
# Opcional: re-referenciar las poses al joint de la pelvis para una vista más estable
J_rel = J - J[:, [0], :]

# Utilidad: ejes iguales en 3D
def set_axes_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()
    x_range = abs(x_limits[1] - x_limits[0])
    y_range = abs(y_limits[1] - y_limits[0])
    z_range = abs(z_limits[1] - z_limits[0])
    max_range = max([x_range, y_range, z_range])
    x_middle = np.mean(x_limits); y_middle = np.mean(y_limits); z_middle = np.mean(z_limits)
    ax.set_xlim3d([x_middle - max_range/2, x_middle + max_range/2])
    ax.set_ylim3d([y_middle - max_range/2, y_middle + max_range/2])
    ax.set_zlim3d([z_middle - max_range/2, z_middle + max_range/2])

# Número de frames a visualizar
n_frames = min(100, J_rel.shape[0])

for t in range(n_frames):
    xs, ys, zs = J_rel[t].T  # (49,), (49,), (49,)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')

    # Puntos 3D (no especificamos colores)
    ax.scatter(xs, ys, zs, s=12)

    # Vista y etiquetas
    ax.set_title(f'VIBE joints3d — frame {t}')
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    ax.view_init(elev=15, azim=-70)

    # Ejes iguales para que no se deforme la figura
    set_axes_equal(ax)
    plt.tight_layout()
    plt.show()


In [ ]:
# Visualización detallada del esqueleto de VIBE usando solo las 24 articulaciones SMPL (0..23)
# Se añaden etiquetas con los índices de cada joint para documentar el mapeo usado en el proyecto.

import joblib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # necesario para proyecciones 3D en Matplotlib

out = joblib.load('output/video_salida/vibe_output.pkl')
#out = joblib.load('/content/vibe_output.pkl')

tid = next(iter(out.keys()))
J = out[tid]['joints3d']            # (T, 49, 3)

# Índices de las 24 articulaciones SMPL que usaremos como base para construir el esqueleto VIBE
# --- Usa solo los 24 joints de SMPL (0..23) ---
idx_smpl24 = np.arange(24)
J24 = J[:, idx_smpl24, :]           # (T, 24, 3)
# modificado:
# --- Usa solo los 24 joints de SMPL (0..23) ---
#if J.shape[1] < 24:
#    raise ValueError(f"Se esperaban >=24 joints y llegaron {J.shape[1]}")
#J24 = J[:, :24, :].copy()           # (T, 24, 3)  (copia segura)


# --- Centra en pelvis (índice 0) ---
J24 = J24 - J24[:, [0], :]

# --- Reorden/voltea ejes (prueba esta convención primero) ---
# de (x, y, z) -> (x, z, -y)  suele verse más natural en Matplotlib
def transform(P):
    X = P[:, 0]
    Y = P[:, 2]
    Z = -P[:, 1]
    return np.stack([X, Y, Z], axis=1)

# Árbol cinemático SMPL-24 (padres por índice)
#parents = [-1, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 12, 12, 13, 14, 16, 17, 18, 19, 20, 21]
# modificado:---------------------------------------------------------------------------------
# === Construye 'parents' a mano, editable y seguro ===
# Pon aquí los índices que YA viste en tu gráfico numerado.
# Si aún no sabes alguno, déjalo en None (no dibuja esa línea, pero no rompe nada).
NOSE   = 0  # NARIZ
HEAD   = 1  # CUELLO
R_SH   = 2  # HOMBRO DERECHO
R_ELB  = 3  # CODO DERECHO
R_WRI  = 4  # MUÑECA DERECHA
L_SH   = 5  # HOMBRO IZQUIERDO
L_ELB  = 6  # CODO IZQUIERDO
L_WRI  = 7  # MUÑECA IZQUIERDA
PELVIS = 8  # PELVIS
R_HIP  = 9  # CADERA DERECHA
R_KNEE = 10 # RODILLA DERECHA
R_ANK  = 11 # TOBILLO DERECHO
L_HIP  = 12 # CADERA IZQUIERDA
L_KNEE = 13 # RODILLA IZQUIERDA
L_ANK  = 14 # TOBILLO IZQUIERDO
R_EYE  = 15 # OJO DERECHO
L_EYE  = 16 # OJO IZQUIERDO
R_EAR  = 17 # OREJA DERECHA
L_EAR  = 18 # OREJA IZQUIERDA
L_FT_1 = 19 # PIE IZQUIERDO INTERNO
L_FT_2 = 20 # PIE IZQUIERDO EXTERNO
R_FT_1 = 22 # PIE DERECHO INTERNO
R_FT_2 = 23 # PIE DERECHO EXTERNO

# El 21 es un punto en el pie pero parece redundante

# Inicializa todos sin padre
parents = [-1]*24

def set_parent(child, parent):
    """Asigna 'parent' a 'child' solo si ambos índices son válidos.
       Si alguno es None, simplemente no hace nada (no rompe)."""
    if child is None or parent is None:
        return
    if not (0 <= child < 24 and 0 <= parent < 24):
        raise ValueError(f"Índice fuera de rango: child={child}, parent={parent}")
    parents[child] = parent

# --- Rellena SOLO lo que ya sabes (puedes correr así y ver un esqueleto parcial) ---

# Pierna izquierda
set_parent(L_HIP,  PELVIS)   # L_hip   <- pelvis
set_parent(L_KNEE, L_HIP)    # L_knee  <- L_hip
set_parent(L_ANK,  L_KNEE)   # L_ankle <- L_knee
set_parent(L_FT_1, L_ANK)    # L_foot1 <- L_ankle
set_parent(L_FT_2, L_ANK)   # L_foot2 <- L_ankle

# Pierna derecha
set_parent(R_HIP,  PELVIS)   # R_hip   <- pelvis
set_parent(R_KNEE, R_HIP)    # R_knee  <- R_hip
set_parent(R_ANK,  R_KNEE)   # R_ankle <- R_knee
set_parent(R_FT_1, R_ANK)    # L_foot1 <- L_ankle
set_parent(R_FT_2, R_ANK)   # L_foot2 <- L_ankle

# Torso
set_parent(R_SH, HEAD)         # Head  <- R_SH
set_parent(L_SH, HEAD)         # Head  <- L_SH
set_parent(HEAD, PELVIS)       # Pelvis  <- Head

# Brazos
set_parent(R_ELB, R_SH)      # R_elbow <- R_shoulder
set_parent(R_WRI, R_ELB)     # R_wrist <- R_elbow
set_parent(L_ELB, L_SH)      # L_elbow <- L_shoulder
set_parent(L_WRI, L_ELB)     # L_wrist <- L_elbow

# Cara
set_parent(NOSE,  HEAD)      # Head  <- Nose
set_parent(R_EYE, NOSE)      # Nose <- R_shoulder
set_parent(L_EYE, NOSE)      # Nose <- L_shoulder
set_parent(R_EAR, R_EYE)      # Nose <- R_shoulder
set_parent(L_EAR, L_EYE)      # Nose <- L_shoulder

#---------------------------------------------------------------------------------------------


def set_axes_equal(ax):
    xl, zl, yl = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
    r = max(xl[1]-xl[0], yl[1]-yl[0], zl[1]-zl[0])
    cx, cy, cz = np.mean(xl), np.mean(yl), np.mean(zl)
    ax.set_xlim(cx-r/2, cx+r/2)
    ax.set_ylim(cy-r/2, cy+r/2)
    ax.set_zlim(cz-r/2, cz+r/2)

# modificado:
#def set_axes_equal(ax):
#    xl, yl, zl = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
#    r = max(xl[1]-xl[0], yl[1]-yl[0], zl[1]-zl[0])
#    cx, cy, cz = np.mean(xl), np.mean(yl), np.mean(zl)
#    ax.set_xlim(cx - r/2, cx + r/2)
#    ax.set_ylim(cy - r/2, cy + r/2)
#    ax.set_zlim(cz - r/2, cz + r/2)


n_frames = min(100, J24.shape[0])
for t in range(n_frames):
    P = transform(J24[t])           # (24, 3) ya transformado

    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111, projection='3d')

    # Dibuja esqueleto SMPL-24
    for j in range(24):
        p = parents[j]
        if p >= 0:
            ax.plot([P[j,0], P[p,0]],
                    [P[j,1], P[p,1]],
                    [P[j,2], P[p,2]])
    # modificado:
    # Dibuja esqueleto SMPL-24 con aristas correctas
    #for a, b in EDGES_SMPL24:
    #    ax.plot([P[a,0], P[b,0]],
    #            [P[a,1], P[b,1]],
    #            [P[a,2], P[b,2]])


    # Puntos (opcional)
    ax.scatter(P[:,0], P[:,1], P[:,2], s=12)
    # <<< NUEVO: etiquetas con índice >>>------------------------------------------------
    for i, (x,y,z) in enumerate(P):
      ax.text(x, y, z, str(i), fontsize=8, color="red")


    ax.set_title(f'VIBE — frame {t}')
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    ax.view_init(elev=15, azim=-70)
    set_axes_equal(ax)
    plt.tight_layout()
    plt.show()


In [ ]:
# === Visualizar un frame específico ===
# Usa las variables globales J24 (poses 3D, 24 joints) y parents (conexiones del esqueleto) definidas en celdas anteriores.
# Permite inspeccionar en detalle un solo frame con los índices de cada articulación.

# Selecciona el índice de frame que quieres visualizar
t = 300

P = J24[t].copy()  # (24,3)
try: P = transform(P)     # si definiste transform()
except NameError: pass

import numpy as np, matplotlib.pyplot as plt

def set_axes_equal(ax):
    xl = ax.get_xlim3d(); yl = ax.get_ylim3d(); zl = ax.get_zlim3d()
    xmid = np.mean(xl); ymid = np.mean(yl); zmid = np.mean(zl)
    r = max(xl[1]-xl[0], yl[1]-yl[0], zl[1]-zl[0])
    ax.set_xlim3d(xmid - r/2, xmid + r/2)
    ax.set_ylim3d(ymid - r/2, ymid + r/2)
    ax.set_zlim3d(zmid - r/2, zmid + r/2)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection='3d')

# huesos (si tienes 'parents')
if 'parents' in globals():
    for j,p in enumerate(parents):
        if p >= 0:
            ax.plot([P[j,0], P[p,0]], [P[j,1], P[p,1]], [P[j,2], P[p,2]])

# puntos + IDs
ax.scatter(P[:,0], P[:,1], P[:,2], s=14)
for i,(x,y,z) in enumerate(P):
    ax.text(x, y, z, str(i), fontsize=8)

ax.set_title(f'VIBE — frame {t}')
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
ax.view_init(elev=15, azim=-70)
set_axes_equal(ax)
plt.tight_layout(); plt.show()

In [ ]:
# === Exportar TODAS las poses a PNG y construir MP4 con misma duración que el video original ===
# Genera un PNG por frame con el esqueleto 3D y construye un video (MP4) sincronizado con el video original.
# Útil para incluir ejemplos visuales en la tesis; no se usa directamente en el cálculo de métricas.
import os, glob, subprocess, sys
import numpy as np
import matplotlib.pyplot as plt

# (opcional pero recomendado) pon aquí la ruta del video original para leer su FPS:
# Ruta del video original para extraer el FPS y respetar la duración al crear el nuevo MP4
ORIGINAL_VIDEO = '/content/video_salida.mp4'  # <- cámbialo si tu ruta es otra; si no, intenta sin esto

# carpeta para frames y salida
SAVE_DIR = 'output/skeleton_frames_auto_all'
OUT_MP4  = 'output/skeleton_smpl.mp4'
os.makedirs(SAVE_DIR, exist_ok=True)

# 1) determinar FPS del video original (si está disponible); fallback a 25
fps = 25
try:
    import cv2
    if ORIGINAL_VIDEO and os.path.exists(ORIGINAL_VIDEO):
        cap = cv2.VideoCapture(ORIGINAL_VIDEO)
        fps_read = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        if fps_read and fps_read > 0:
            fps = int(round(fps_read))
            print(f"FPS detectado del video original: {fps}")
        else:
            print("No se pudo leer FPS del video; uso 25.")
    else:
        print("No se encontró el video original; uso 25 FPS.")
except Exception as e:
    print("OpenCV no disponible para leer FPS; uso 25 FPS.", e)

# 2) renderizar TODOS los frames disponibles
n_frames_vid = int(J24.shape[0])
print("Total de frames a renderizar:", n_frames_vid)

for t in range(n_frames_vid):
    P = transform(J24[t])   # (24,3)

    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111, projection='3d')

    # líneas según 'parents' (no tocamos tu lógica)
    for j in range(24):
        p = parents[j]
        if p >= 0:
            ax.plot([P[j,0], P[p,0]],
                    [P[j,1], P[p,1]],
                    [P[j,2], P[p,2]])

    # puntos + índices (si no quieres los números, comenta el for)
    ax.scatter(P[:,0], P[:,1], P[:,2], s=12)
    for i,(x,y,z) in enumerate(P):
        ax.text(x,y,z,str(i),fontsize=8,color="red")

    ax.set_title(f'VIBE — frame {t}')
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    ax.view_init(elev=15, azim=-70)
    set_axes_equal(ax)
    plt.tight_layout()

    # guardar PNG
    fig.savefig(f"{SAVE_DIR}/frame_{t:05d}.png", dpi=120)
    plt.close(fig)

# 3) construir MP4 con el mismo FPS del original (duración = n_frames / fps)
try:
    subprocess.run([
        'ffmpeg','-y',
        '-framerate', str(fps),
        '-i', f'{SAVE_DIR}/frame_%05d.png',
        '-c:v','libx264','-pix_fmt','yuv420p',
        OUT_MP4
    ], check=True)
    print(f"MP4 listo con ffmpeg: {OUT_MP4}  (duración ~ {n_frames_vid/fps:.2f} s)")
except Exception as e:
    print("ffmpeg no disponible o falló; usando imageio como respaldo...", e)
    import imageio.v2 as imageio
    frames = sorted(glob.glob(f'{SAVE_DIR}/frame_*.png'))
    w = imageio.get_writer(OUT_MP4, fps=fps)
    for f in frames:
        w.append_data(imageio.imread(f))
    w.close()
    print(f"MP4 listo con imageio: {OUT_MP4}  (duración ~ {n_frames_vid/fps:.2f} s)")

import shutil
from google.colab import files

# Comprimir la carpeta a ZIP
zip_path = shutil.make_archive("skeleton_frames_auto_all", 'zip', "output/skeleton_frames_auto_all")

# Descargar el ZIP
files.download(zip_path)


<!-- Fin de la sección de visualización de VIBE -->


In [ ]:
# Comprobación rápida de que /content/vibe_output.pkl existe y tiene joints3d con las 24 articulaciones SMPL
# (este archivo se usará en otros notebooks para convertir a 14 articulaciones y calcular métricas)
import joblib
import numpy as np

# Cargar PKL
out = joblib.load('/content/vibe_output.pkl')

tid = next(iter(out.keys()))
J = out[tid]['joints3d']      # (T, 49, 3)

# Filtrar a las 24 articulaciones SMPL (0..23)
J24 = J[:, :24, :]            # (T, 24, 3)

print("Forma del array:", J24.shape)
print(J24[10])                 # coordenadas del primer frame
